# Visualization of training and results

In [70]:
############
# Packages #
############
import os
import sys
import numpy as np
import pickle
from pathlib import Path
import plotly.io as pio
from prettytable import PrettyTable
import itertools

##################
#      Imports   #
##################
root_path = Path(os.getcwd())
# root_path should end by \nonlinear_ICA:
print(root_path)
sys.path.insert(0, str(root_path))
pio.renderers.default = "plotly_mimetype+notebook"

##############
#    Paths   #
##############
get_trainpath = lambda i: root_path.joinpath("models", "TCL", f"TCL_{i}")
get_respath = lambda i,j: root_path.joinpath("models", "classif", f"classif_{j}", "dataset_1", f"TCL_{i}", "NORM")

################
#   runs ids   #
################

TCL_ids = [1,2,3]
classif_ids = {
    1:[1], 
    2:[], 
    3:[]
}

#############
#    Load   #
#############
load_pkl = lambda path, name: pickle.load(open(str(path.joinpath(f"{name}.pkl")), 'rb'))

#############
#    Plot   #
#############

init_tab = lambda dct, lst: PrettyTable(list(dct.keys())+lst)
init_tab2 = lambda lst_dct: PrettyTable(  ["TCL","cl"]+list(itertools.chain.from_iterable([list(dct.keys()) for dct in lst_dct]))   )
list_val = lambda dct: list(dct.values())
list_valround = lambda dct: np.round(list(dct.values()),3).squeeze().tolist()



C:\Users\samud\Bureau\Python code\MVA\Advanced ML\nonlinear_ICA


The following script gather results into tables. Please use an LLM to convert it into latex format.

## Training

In [68]:
start = True
for i in TCL_ids:
    params = load_pkl(get_trainpath(i),"params")
    train_loss =  load_pkl(get_trainpath(i),"model_loss")
    if start:
        tab = init_tab(params, ["accuracy", "f1"])
        start = False
    tab.add_row(list_val(params)+list(np.round(train_loss,3)))

print(tab)

+------------+---------------+------------+-----------+-----------+-------+----------+-------+
| num_layers | hidden_layers | activation | input_dim | pool_size | slope | accuracy |   f1  |
+------------+---------------+------------+-----------+-----------+-------+----------+-------+
|     1      |      [1]      | ['lrelu']  |     12    |     1     |  0.1  |  2.452   | 2.448 |
|     1      |      [3]      | ['lrelu']  |     12    |     1     |  0.1  |  2.359   | 2.357 |
|     1      |      [5]      | ['lrelu']  |     12    |     1     |  0.1  |  2.351   | 2.343 |
+------------+---------------+------------+-----------+-----------+-------+----------+-------+


## Results

In [71]:
start = True
for i in TCL_ids:
    for j in classif_ids[i]:
        best_params = load_pkl(get_respath(i,j),"best_params")
        res_best_model =  load_pkl(get_respath(i,j),"res_best_model")
                
        results_gs_SVM = load_pkl(get_respath(i,j),"results_gs_SVM")
        del results_gs_SVM["param_svm__C"]
        del results_gs_SVM["param_svm__gamma"]
        del results_gs_SVM["param_svm__kernel"]
        del results_gs_SVM["params"] #doublons
        if start:
            tab_train = init_tab2([best_params, res_best_model["train"]])
            tab_test = init_tab2([best_params, res_best_model["test"]])
            tab_svm = init_tab2([results_gs_SVM])
            strat = False
        tab_train.add_row([i,j] + list_val(best_params)+ list_valround(res_best_model["train"]))
        tab_test.add_row([i,j] + list_val(best_params)+ list_valround(res_best_model["test"]))
        tab_svm.add_row([i,j] + list_valround(results_gs_SVM))

print("\nTRAIN \n",tab_train)
print("\nTEST \n",tab_test)
print("\nSVM_STAT \n",tab_svm)


TRAIN 
 +-----+----+--------+------------+-------------+----------+----------+--------+-----------+---------+
| TCL | cl | svm__C | svm__gamma | svm__kernel | accuracy | f1_score | recall | precision | roc_auc |
+-----+----+--------+------------+-------------+----------+----------+--------+-----------+---------+
|  1  | 1  |  0.1   |   scale    |    linear   |  0.628   |  0.604   | 0.651  |   0.564   |  0.676  |
+-----+----+--------+------------+-------------+----------+----------+--------+-----------+---------+

TEST 
 +-----+----+--------+------------+-------------+----------+----------+--------+-----------+---------+
| TCL | cl | svm__C | svm__gamma | svm__kernel | accuracy | f1_score | recall | precision | roc_auc |
+-----+----+--------+------------+-------------+----------+----------+--------+-----------+---------+
|  1  | 1  |  0.1   |   scale    |    linear   |  0.597   |  0.567   | 0.604  |   0.535   |  0.643  |
+-----+----+--------+------------+-------------+----------+------